<a href="https://colab.research.google.com/github/atlas-github/iproperty_scraper/blob/master/iproperty_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

cookies = {
    'JSESSIONID': '44135523471d5c41',
}

headers = {
    'authority': 'www.google-analytics.com',
    'upgrade-insecure-requests': '1',
    'dnt': '1',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36',
    'sec-fetch-dest': 'image',
    'accept': 'image/webp,image/apng,image/*,*/*;q=0.8',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-user': '?1',
    'referer': 'https://www.iproperty.com.my/property/findanagent.aspx?ty=as&ak=&rk=&rmp=10&pg=1&st=&ct=&st1=&ct1=',
    'accept-language': 'en-US,en;q=0.9,ms;q=0.8',
    'cookie': 'sb=ttOLXmWVjU_AYqwXKGqExKjv; datr=ttOLXjUYHZIiDqQeh-h3VLYP; locale=en_US; c_user=641411155; xs=186%3AXzqcA55KWV8g-w%3A2%3A1586222035%3A10708%3A6401; spin=r.1001951938_b.trunk_t.1586222035_s.1_v.2_; ; presence=EDvF3EtimeF1586237646EuserFA2641411155A2EstateFDt3F_5b_5dElm3FA2user_3a249944268675281A2Eutc3F1586237645964G586237646038CEchF_7bCC; wd=1268x959; fr=1C5FzrVgkEyjIIZ8Y.AWWGNSv2kT73foro1R9v5OXbueE.Bei9O2.1Z.F6L.0.0.BejFWm.AWVrWLuL',
    'Referer': 'https://www.iproperty.com.my/property/findanagent.aspx?ty=as&ak=&rk=&rmp=10&pg=1&st=&ct=&st1=&ct1=',
    'Sec-Fetch-Dest': 'empty',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36',
    'DNT': '1',
    'Origin': 'https://www.iproperty.com.my',
    'if-none-match': '"073b35c69744014f5d2406710e188e97"',
    'if-modified-since': 'Mon, 18 Mar 2019 15:45:52 GMT',
    'Connection': 'keep-alive',
    'Accept': '*/*',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Mode': 'no-cors',
    'Accept-Language': 'en-US,en;q=0.9,ms;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'content-type': 'text/plain',
    'origin': 'https://www.iproperty.com.my',
    'Content-Length': '0',
    'Content-Type': 'text/plain;charset=UTF-8',
}

data = {
  'Q_LOC': 'https://www.iproperty.com.my/property/findanagent.aspx?ty=as&ak=&rk=&rmp=10&pg=1&st=&ct=&st1=&ct1='
}

name_clean = []
phone_clean = []
email_clean = []
website_clean = []
listings_clean = []

#response = requests.post('https://www.iproperty.com.my/property/findanagent.aspx', headers=headers, params=params, cookies=cookies, data=data)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".

In [0]:
for i in range(1, 1105):
    
    params = (
    ('ty', 'as'),
    ('ak', ''),
    ('rk', ''),
    ('rmp', '10'),
    ('pg', str(i)),
    ('st', ''),
    ('ct', ''),
    ('st1', ''),
    ('ct1', ''),
    )
    
    response = requests.post('https://www.iproperty.com.my/property/findanagent.aspx?ty=as&ak=&rk=&rmp=10&pg='+str(i)+'&st=&ct=&st1=&ct1=', headers=headers, cookies=cookies, data=data)
    soup = BeautifulSoup(response.text, "html.parser")

    name = soup.findAll('a', {'class': 'BlueBold14-1'})
    phone = soup.findAll('a', {'class': 'csagentphonelead'})
    email = soup.findAll('a', {'class': 'csagentemaillead'})
    website = soup.findAll('a', {'class': 'csweblead'})
    listings = soup.findAll('a', {'class': 'cslistinglead'})

    name_list = list(name)
    phone_list = list(phone)
    email_list = list(email)
    website_list = list(website)
    listings_list = list(listings)

    for i in range(len(name_list)):
        name_clean.append(name_list[i].get_text())

    for i in range(len(phone_list)):
        phone_clean.append(phone_list[i]['data'])

    for i in range(len(email_list)):
        email_clean.append(email_list[i]['data'])

    for i in range(len(website_list)):
        website_clean.append(website_list[i]['href'])

    for i in range(len(listings_list)):
        listings_clean.append(listings_list[i].get_text())

In [0]:
df = pd.DataFrame(zip(name_clean, phone_clean, email_clean, website_clean, listings_clean))
df = df.rename(columns = {0: "Name", 1: "Phone", 2: "Email", 3: "Website", 4: "Listings"})
df

In [0]:
df.to_csv("iproperty.csv")

In [0]:
df2 = df.loc[df['Website'] != 'http://.iagent.my']
df2

In [0]:
website_identify = []
ren_list = []

for i in range(0, len(df2)):
    response2 = requests.post(df2.iloc[i, 3])
    soup2 = BeautifulSoup(response2.text, "html.parser")
    ren_number = soup2.findAll('span', {'class': 'bodytext'})
    try:
        ren_list.append(ren_number[2].get_text())
    except:
        ren_list.append("Not Available")
    website_identify.append(df.iloc[i, 3])

In [0]:
website_identify

In [0]:
ren_list

In [0]:
len(ren_list)

In [0]:
df3 = pd.DataFrame(zip(website_identify, ren_list))
df3 = df3.rename(columns = {0: "Website", 1: "REN Number"})
df3.to_csv("ren_number.csv")